pip install google-play-scraper AND 
pip install app_store_scraper

# Data extraction

## Step 1: Classification training/testing Machine Learning dataset

Retrieving data from the 3 main feedback platforms :

    Google Play
    Apple Store
    Trustpilot

Then storing of the reviews of each platform into 1 csv doc.

Installation of pip install google-play-scraper to scrap app reviews

In [ ]:
!pip install google-play-scraper
!pip install app_store_scraper

     |████████████████████████████████| 52 kB 622 kB/s  eta 0:00:01
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.2-py3-none-any.whl size=24393 sha256=d5f8c4d483b1f45f160256e72227922496332089d7cde526ac077f05c5d58144
  Stored in directory: /root/.cache/pip/wheels/98/99/eb/bbb9d24a5c526980647efc10336eaaeffcf07749f581111128
Successfully built google-play-scraper


In [2]:
# Import relevant libraries and packages
import pandas as pd
import numpy as np
import pickle

import requests
from bs4 import BeautifulSoup
import json

from google_play_scraper import app
from google_play_scraper import Sort, reviews_all
from app_store_scraper import AppStore
import snscrape.modules.twitter as sntwitter
import pandas as pd

ModuleNotFoundError: No module named 'snscrape'

## **1. Google Play**

google app id => "com.app.tgtg"

In [ ]:
# Retrieving app reviews from google play 
reviews_g = reviews_all(
    'com.app.tgtg',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='uk', # 'uk'
    sort=Sort.NEWEST, # since default is to Sort.MOST_RELEVANT
)

In [ ]:
# Converting reviews in pandas dataframe
df_google = pd.DataFrame(np.array(reviews_g),columns=['review'])
df_google = df_google.join(pd.DataFrame(df_google.pop('review').tolist()))
df_google.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOHKjUjEAjZuuQvBSTb_0-c7LG--AdBELjURUU_...,Why in the world am I allowed such a long user...,https://play-lh.googleusercontent.com/a/AATXAJ...,It does exactly what it says it does efficient...,5,0,21.11.1,2021-12-11 08:03:30,None,NaT
1,gp:AOqpTOGcsoUhTFPKgN8WPsk_Tdd2vRlvTzkSXNBDiZI...,cathyhammer,https://play-lh.googleusercontent.com/a-/AOh14...,"Well, I've only done 2 different vendors. The ...",4,0,21.11.1,2021-12-11 04:56:39,None,NaT
2,gp:AOqpTOHPLWFKIrKg2pszn9ry43bgMQh_PxeAOGXFNsx...,Carl Sager,https://play-lh.googleusercontent.com/a-/AOh14...,Still in my first week. And I'm looking forwar...,5,0,21.11.1,2021-12-11 03:23:46,None,NaT
3,gp:AOqpTOFVAXjBTnAAHreROAhOFymJyRmKQ8GIYysY6N_...,LDJCR7,https://play-lh.googleusercontent.com/a-/AOh14...,loving the app.,5,0,21.11.1,2021-12-10 22:52:33,None,NaT
4,gp:AOqpTOGn2A6aX-KdnukiV7VKjuHDKRp4RvVLhGxyPd-...,Rosalinda Martin,https://play-lh.googleusercontent.com/a/AATXAJ...,AT FIRST: 1) Lovely experience. charming staff...,2,1,21.11.1,2021-12-10 22:22:27,None,NaT


In [ ]:
# Storing raw data in pickle format
df_google.to_pickle('GooglePlayReview_TooGoodToGo.pkl')

In [3]:
# Importing doc back to standardize it into workable format
df_google = "data/GooglePlayReview_TooGoodToGo.pkl" #path to be written

with open(df_google, 'rb') as f:
  df_google = pickle.load(f)

df_google.head(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOHKjUjEAjZuuQvBSTb_0-c7LG--AdBELjURUU_...,Why in the world am I allowed such a long user...,https://play-lh.googleusercontent.com/a/AATXAJ...,It does exactly what it says it does efficient...,5,0,21.11.1,2021-12-11 08:03:30,None,NaT
1,gp:AOqpTOGcsoUhTFPKgN8WPsk_Tdd2vRlvTzkSXNBDiZI...,cathyhammer,https://play-lh.googleusercontent.com/a-/AOh14...,"Well, I've only done 2 different vendors. The ...",4,0,21.11.1,2021-12-11 04:56:39,None,NaT
2,gp:AOqpTOHPLWFKIrKg2pszn9ry43bgMQh_PxeAOGXFNsx...,Carl Sager,https://play-lh.googleusercontent.com/a-/AOh14...,Still in my first week. And I'm looking forwar...,5,0,21.11.1,2021-12-11 03:23:46,None,NaT
3,gp:AOqpTOFVAXjBTnAAHreROAhOFymJyRmKQ8GIYysY6N_...,LDJCR7,https://play-lh.googleusercontent.com/a-/AOh14...,loving the app.,5,0,21.11.1,2021-12-10 22:52:33,None,NaT
4,gp:AOqpTOGn2A6aX-KdnukiV7VKjuHDKRp4RvVLhGxyPd-...,Rosalinda Martin,https://play-lh.googleusercontent.com/a/AATXAJ...,AT FIRST: 1) Lovely experience. charming staff...,2,1,21.11.1,2021-12-10 22:22:27,None,NaT


In [4]:
df_google.shape

(19848, 10)

In [5]:
# Dropping unnecessary columns
df_google = df_google.drop(columns=['reviewId','userName', 'userImage', 'thumbsUpCount', 'reviewCreatedVersion','replyContent','repliedAt'])

In [6]:
# Checking dataframe
df_google.head(5)

,content,score,at
0,It does exactly what it says it does efficient...,5,2021-12-11 08:03:30
1,"Well, I've only done 2 different vendors. The ...",4,2021-12-11 04:56:39
2,Still in my first week. And I'm looking forwar...,5,2021-12-11 03:23:46
3,loving the app.,5,2021-12-10 22:52:33
4,AT FIRST: 1) Lovely experience. charming staff...,2,2021-12-10 22:22:27


In [7]:
# Renaming columns
df_google.columns=['review_content','rating','date']

In [8]:
# Adding "source" column to put "Google"
df_google['source'] = 'Google'

In [9]:
# Checking dataframe
df_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19848 entries, 0 to 19847
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   review_content  19848 non-null  object        
 1   rating          19848 non-null  int64         
 2   date            19848 non-null  datetime64[ns]
 3   source          19848 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 620.4+ KB


In [10]:
# Formatting date
df_google['date'] = pd.to_datetime(df_google['date'], format='%Y-%m-%d %Hh%Mm%Ssec')

In [11]:
# Check new format
df_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19848 entries, 0 to 19847
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   review_content  19848 non-null  object        
 1   rating          19848 non-null  int64         
 2   date            19848 non-null  datetime64[ns]
 3   source          19848 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 620.4+ KB


In [12]:
# Formatting rating
df_google['rating'].astype(int)

0        5
1        4
2        5
3        5
4        2
        ..
19843    5
19844    4
19845    5
19846    5
19847    5
Name: rating, Length: 19848, dtype: int64

In [13]:
# Check new format
df_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19848 entries, 0 to 19847
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   review_content  19848 non-null  object        
 1   rating          19848 non-null  int64         
 2   date            19848 non-null  datetime64[ns]
 3   source          19848 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 620.4+ KB


In [14]:
# Checking ISNA
df_google.isna().sum()

review_content    0
rating            0
date              0
source            0
dtype: int64

In [15]:
# Checking duplicates 
df_google.duplicated().sum()

0

## **2. Apple Store**

apple store app name => too-good-to-go-end-food-waste AND its id => 1060683933

In [ ]:
# Retrieving app reviews from apple store
apple_r = AppStore(country="gb", app_name="too-good-to-go-end-food-waste",app_id=1060683933)
apple_r.review(how_many=10000)

2021-12-11 09:27:15,061 [INFO] Base - Initialised: AppStore('gb', 'too-good-to-go-end-food-waste', 1060683933)
2021-12-11 09:27:15,062 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/gb/app/too-good-to-go-end-food-waste/id1060683933
2021-12-11 09:27:34,515 [ERROR] Base - Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/gb/apps/1060683933/reviews?l=en-GB&offset=140&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 429 error responses'))
2021-12-11 09:27:34,517 [INFO] Base - [id:1060683933] Fetched 140 reviews (140 fetched in total)


In [ ]:
# Converting format (JSON to Pandas Dataframe)
apple = pd.DataFrame(np.array(apple_r.reviews),columns=['review'])
df_apple = apple.join(pd.DataFrame(apple.pop('review').tolist()))
df_apple.head()

,title,isEdited,review,userName,date,rating,developerResponse
0,Not that great to go more like,False,I have had this app for a while but it’s took ...,Jay shake,2021-02-27 15:09:54,5,NaN
1,Barley Farm - Eccles,False,This was the first time using the app. I was a...,Ck1TWO,2020-09-28 21:50:36,5,NaN
2,"Good idea in theory, but flawed",False,Just got the app really so I don’t have much t...,AnActualPoolPlayer,2019-09-07 11:44:36,1,"{'id': 10585622, 'body': 'Hi there, We're sor..."
3,Outstanding!,False,Used the app for the first time today and was ...,Ceilidh279,2019-09-07 17:21:51,5,NaN
4,Mixed review,False,"I like this idea, sometimes we get some great ...",mobilebob2,2021-11-04 16:43:02,5,NaN


In [ ]:
# Storing raw data in pickle format
df_apple.to_pickle('AppleStoreReview_TooGoodToGo.pkl')

In [16]:
# Importing doc back to standardize it into workable format
df_apple = "data/AppleStoreReview_TooGoodToGo.pkl" #path to be written

with open(df_apple, 'rb') as f:
    df_apple = pickle.load(f)

df_apple.head(5)

,title,isEdited,review,userName,date,rating,developerResponse
0,Not that great to go more like,False,I have had this app for a while but it’s took ...,Jay shake,2021-02-27 15:09:54,5,NaN
1,Barley Farm - Eccles,False,This was the first time using the app. I was a...,Ck1TWO,2020-09-28 21:50:36,5,NaN
2,"Good idea in theory, but flawed",False,Just got the app really so I don’t have much t...,AnActualPoolPlayer,2019-09-07 11:44:36,1,"{'id': 10585622, 'body': 'Hi there, We're sor..."
3,Outstanding!,False,Used the app for the first time today and was ...,Ceilidh279,2019-09-07 17:21:51,5,NaN
4,Mixed review,False,"I like this idea, sometimes we get some great ...",mobilebob2,2021-11-04 16:43:02,5,NaN


In [17]:
df_apple.shape

(2000, 7)

In [18]:
# to concatenate Title & review content
df_apple['review_content'] = df_apple['title'] + ' ' + df_apple['review']

In [19]:
# Dropping unnecessary columns
df_apple = df_apple.drop(columns=['review','userName', 'isEdited','title','developerResponse'])

In [20]:
# Checking dataframe
df_apple.head(5)

,date,rating,review_content
0,2021-02-27 15:09:54,5,Not that great to go more like I have had this...
1,2020-09-28 21:50:36,5,Barley Farm - Eccles This was the first time u...
2,2019-09-07 11:44:36,1,"Good idea in theory, but flawed Just got the a..."
3,2019-09-07 17:21:51,5,Outstanding! Used the app for the first time t...
4,2021-11-04 16:43:02,5,"Mixed review I like this idea, sometimes we ge..."


In [21]:
# Reordering columns
df_apple = df_apple[['review_content', 'rating', 'date']]

In [22]:
# Checking dataframe
df_apple.head(5)

,review_content,rating,date
0,Not that great to go more like I have had this...,5,2021-02-27 15:09:54
1,Barley Farm - Eccles This was the first time u...,5,2020-09-28 21:50:36
2,"Good idea in theory, but flawed Just got the a...",1,2019-09-07 11:44:36
3,Outstanding! Used the app for the first time t...,5,2019-09-07 17:21:51
4,"Mixed review I like this idea, sometimes we ge...",5,2021-11-04 16:43:02


In [23]:
# Adding "source" column to put "Apple"
df_apple['source'] = 'Apple'

In [24]:
# Formatting date
df_apple['date'] = pd.to_datetime(df_apple['date'], format='%Y-%m-%d %Hh%Mm%Ssec')

In [25]:
# Check new format
df_apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   review_content  2000 non-null   object        
 1   rating          2000 non-null   int64         
 2   date            2000 non-null   datetime64[ns]
 3   source          2000 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 62.6+ KB


In [26]:
# Checking ISNA
df_apple.isna().sum()

review_content    0
rating            0
date              0
source            0
dtype: int64

In [27]:
# Checking duplicates 
df_apple.duplicated().sum()

0

## **3. Trustpilot**

In [ ]:
# Retrieving app reviews from trustpilot website
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0'
}
response = requests.get('https://uk.trustpilot.com/review/toogoodtogo.com?page=2', headers=headers)
response.status_code

200

In [ ]:
soup = BeautifulSoup(response.content)
data = json.loads(soup.find('script', type='application/json').string)
reviews_t = data["props"]["pageProps"]["reviews"]

sample_reply = reviews_t[0]
sample_reply

{'consumer': {'countryCode': 'GB',
  'displayName': 'Yvonne Simpson',
  'hasReviewFacebookFriend': False,
  'id': '61914c343f59d00012152fb3',
  'imageUrl': '',
  'isDirectFacebookFriend': False,
  'numberOfReviews': 1},
 'consumersReviewCountOnSameDomain': 1,
 'consumersReviewCountOnSameLocation': 0,
 'dates': {'experiencedDate': None,
  'publishedDate': '2021-11-14T17:49:53.000Z',
  'updatedDate': '2021-11-14T17:50:30.000Z'},
 'filtered': False,
 'hasUnhandledReports': False,
 'id': '61914c4101034f34babf3c56',
 'labels': {'merged': None,
  'verification': {'createdDateTime': '2021-11-14T17:49:53.000Z',
   'isVerified': False,
   'reviewSourceName': 'Organic',
   'verificationLevel': 'not-verified',
   'verificationSource': 'invitation'}},
 'language': 'en',
 'likes': 0,
 'location': None,
 'productReviews': [],
 'rating': 1,
 'reply': {'message': "Hi Yvonne,\n\nThanks for getting in touch and for fighting food waste with us.\n\nWe'd love to chat about your experience. Please get in to

In [ ]:
#collecting all the info of the review we might need to process after checking it
reviewsText = []
reviewsTitle = []
ratings = []
country = []
language = []
publishedDate = []


for i in range(1,75):
    url = "https://uk.trustpilot.com/review/toogoodtogo.com?page={}".format(i)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content)
    data = json.loads(soup.find('script', type='application/json').string)
    reviews_t = data["props"]["pageProps"]["reviews"]
    for review in reviews_t:
        reviewsTitle.append(review['title'])
        reviewsText.append(review['text'])
        ratings.append(review['rating'])
        country.append(review['consumer']['countryCode'])
        language.append(review['language'])
        publishedDate.append(review['dates']['publishedDate'])

In [ ]:
# Converting reviews in pandas dataframe
df_trust = pd.DataFrame(np.column_stack([ratings, country, publishedDate, reviewsTitle, reviewsText, language]), 
                               columns=['rating', 'location', 'date', 'title', 'content', 'language'])

In [ ]:
df_trust.head()

,rating,location,date,title,content,language
0,1,GB,2021-12-10T20:35:51.000Z,TOBY CARVERY LATE CANCELLATION,30 minutes before order cancelled..from toby c...,en
1,1,GB,2021-12-08T21:03:08.000Z,Made an order with Co op Erdington,"Made an order with Co op Erdington, it was rea...",en
2,1,NO,2021-12-06T21:25:09.000Z,Feel cheated with the amount of food…,Feel cheated with the amount of food received....,en
3,2,GB,2021-12-06T20:08:14.000Z,X2 orders cancelled and I was charged…,X2 orders cancelled and I was charged for them...,en
4,2,GB,2021-12-06T15:59:02.000Z,"Love the idea, BUT","Love the idea, but out of three times of order...",en


In [ ]:
# Storing raw data in pickle format
df_trust.to_pickle('TrustpilotReview_TooGoodToGo.pkl')

In [28]:
# Importing doc back to standardize it into workable format
df_trust = "data/TrustpilotReview_TooGoodToGo.pkl" 

with open(df_trust, 'rb') as f:
    df_trust = pickle.load(f)

df_trust.head(5)

,rating,location,date,title,content,language
0,1,GB,2021-12-10T20:35:51.000Z,TOBY CARVERY LATE CANCELLATION,30 minutes before order cancelled..from toby c...,en
1,1,GB,2021-12-08T21:03:08.000Z,Made an order with Co op Erdington,"Made an order with Co op Erdington, it was rea...",en
2,1,NO,2021-12-06T21:25:09.000Z,Feel cheated with the amount of food…,Feel cheated with the amount of food received....,en
3,2,GB,2021-12-06T20:08:14.000Z,X2 orders cancelled and I was charged…,X2 orders cancelled and I was charged for them...,en
4,2,GB,2021-12-06T15:59:02.000Z,"Love the idea, BUT","Love the idea, but out of three times of order...",en


In [29]:
# Checking data
df_trust.head(5)

,rating,location,date,title,content,language
0,1,GB,2021-12-10T20:35:51.000Z,TOBY CARVERY LATE CANCELLATION,30 minutes before order cancelled..from toby c...,en
1,1,GB,2021-12-08T21:03:08.000Z,Made an order with Co op Erdington,"Made an order with Co op Erdington, it was rea...",en
2,1,NO,2021-12-06T21:25:09.000Z,Feel cheated with the amount of food…,Feel cheated with the amount of food received....,en
3,2,GB,2021-12-06T20:08:14.000Z,X2 orders cancelled and I was charged…,X2 orders cancelled and I was charged for them...,en
4,2,GB,2021-12-06T15:59:02.000Z,"Love the idea, BUT","Love the idea, but out of three times of order...",en


In [30]:
df_trust.shape

(1480, 6)

In [31]:
# Checking if all the reviews are in english
df_trust['language'].value_counts()

en    1480
Name: language, dtype: int64

In [32]:
df_trust['location'].value_counts()

GB    1120
DK     125
NO      45
US      34
DE      31
NL      31
BE      19
CH      19
FR      17
SE       5
ES       5
AU       4
IT       3
CA       3
GR       3
PL       2
PT       2
MY       2
IE       2
IN       1
AT       1
BN       1
UA       1
RO       1
AW       1
LU       1
HU       1
Name: location, dtype: int64

In [33]:
df_trust = df_trust[df_trust['location']=='GB']

In [34]:
# to concatenate Title & review content
df_trust['review_content'] = df_trust['title'] + ' ' + df_trust['content']

In [35]:
# Checking dataframe
df_trust.head(5)

,rating,location,date,title,content,language,review_content
0,1,GB,2021-12-10T20:35:51.000Z,TOBY CARVERY LATE CANCELLATION,30 minutes before order cancelled..from toby c...,en,TOBY CARVERY LATE CANCELLATION 30 minutes befo...
1,1,GB,2021-12-08T21:03:08.000Z,Made an order with Co op Erdington,"Made an order with Co op Erdington, it was rea...",en,Made an order with Co op Erdington Made an ord...
3,2,GB,2021-12-06T20:08:14.000Z,X2 orders cancelled and I was charged…,X2 orders cancelled and I was charged for them...,en,X2 orders cancelled and I was charged… X2 orde...
4,2,GB,2021-12-06T15:59:02.000Z,"Love the idea, BUT","Love the idea, but out of three times of order...",en,"Love the idea, BUT Love the idea, but out of t..."
5,5,GB,2021-12-06T13:01:07.000Z,"Great idea, keep it up!","I've used the app a few times now, always pick...",en,"Great idea, keep it up! I've used the app a fe..."


In [36]:
# Dropping unnecessary columns
df_trust = df_trust.drop(columns=['location','title','content','language'])

In [37]:
df_trust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1120 entries, 0 to 1467
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   rating          1120 non-null   object
 1   date            1120 non-null   object
 2   review_content  1120 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB


In [38]:
# Formatting date => removing anything on the right of T (T being included)
df_trust['date'] = df_trust['date'].str[:10]

In [39]:
# Formatting date
df_trust['date'] = pd.to_datetime(df_trust['date'], format='%Y-%m-%d')

In [40]:
# Reordering columns
df_trust = df_trust[['review_content', 'rating', 'date']]

In [41]:
# Adding "source" column to put "Trustpilot"
df_trust['source'] = 'Trustpilot'

In [42]:
# Check formatting
df_trust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1120 entries, 0 to 1467
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   review_content  1120 non-null   object        
 1   rating          1120 non-null   object        
 2   date            1120 non-null   datetime64[ns]
 3   source          1120 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 43.8+ KB


In [43]:
# Formatting rating
df_trust['rating'].astype(int)

0       1
1       1
3       2
4       2
5       5
       ..
1454    4
1461    5
1464    5
1465    5
1467    5
Name: rating, Length: 1120, dtype: int64

In [44]:
# Check new format
df_trust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1120 entries, 0 to 1467
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   review_content  1120 non-null   object        
 1   rating          1120 non-null   object        
 2   date            1120 non-null   datetime64[ns]
 3   source          1120 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 43.8+ KB


In [45]:
# Checking ISNA
df_trust.isna().sum()

review_content    0
rating            0
date              0
source            0
dtype: int64

In [46]:
# Checking duplicates 
df_trust.duplicated().sum()

0

## **4. Concatenation**

In [70]:
df = pd.concat([df_google, df_apple, df_trust], axis=0)
df.head(5)

,review_content,rating,date,source
0,It does exactly what it says it does efficient...,5,2021-12-11 08:03:30,Google
1,"Well, I've only done 2 different vendors. The ...",4,2021-12-11 04:56:39,Google
2,Still in my first week. And I'm looking forwar...,5,2021-12-11 03:23:46,Google
3,loving the app.,5,2021-12-10 22:52:33,Google
4,AT FIRST: 1) Lovely experience. charming staff...,2,2021-12-10 22:22:27,Google


In [71]:
df.shape

(22968, 4)

In [72]:
# formatting rating
df['rating'] = df['rating'].astype(int)

In [73]:
# formatting date => to remove day
df['date'] = df['date'].dt.strftime('%Y-%m')

In [74]:
# checking format
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22968 entries, 0 to 1467
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   review_content  22968 non-null  object
 1   rating          22968 non-null  int64 
 2   date            22968 non-null  object
 3   source          22968 non-null  object
dtypes: int64(1), object(3)
memory usage: 897.2+ KB


In [75]:
# formatting date
df['date'] = pd.to_datetime(df['date'], format='%Y-%m')

In [76]:
# checking format
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22968 entries, 0 to 1467
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   review_content  22968 non-null  object        
 1   rating          22968 non-null  int64         
 2   date            22968 non-null  datetime64[ns]
 3   source          22968 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 897.2+ KB


In [77]:
# Storing data in pickle format
df.to_pickle('reviews_tgtg_v0.pkl')

## Step 2: Twitter dataset

In [ ]:
# Creating list to append tweet data to
tweets_list = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#TooGoodToGo since:2021-01-01 until:2021-12-11 lang:en').get_items()):
    if i>5000:
        break
    tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username])
    
# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

In [ ]:
tweets_df

In [ ]:
tweets_df.to_pickle('tweets2021tgtg.pkl')